In [2]:
import pandas as pd
import glob
import os
from shutil import which
import os
import numpy as np
import warnings
import subprocess
warnings.filterwarnings("ignore")

In [24]:
act = []
cgm = []
griggs_act = []
griggs_cgm = []
final_griggs = []
s=[]

wake_sleep_file = 'sleep_time_interval_DG.csv'
number_of_participants = 46
number_of_days = 7
number_of_bands = 7
wake_sleep = pd.read_csv(wake_sleep_file)
for i in range(1,number_of_participants+1):
    act = []
    cgm = []
    griggs_act = []
    griggs_cgm = []
    final_griggs = []
    s=[]    
    for file in glob.glob('Data/CGM Data/*'):        
        num = file.split('/')[-1].split('_')[0]
        
        ################ Read CGM file ###########################################
        if ('00'+str(i) == num) or ('0'+str(i) == num) or (str(i) == num):
            try:
                cgm = pd.read_excel(file)
            except:
                cgm = pd.read_csv(file)

            break          
    if len(cgm) == 0:
        continue
    if any('Unnamed:' in string for string in cgm.columns):
        cgm.columns = cgm.iloc[0]
        cgm = cgm[2:]
 

    if 'Timestamp (YYYY-MM-DDThh:mm:ss)' in cgm.columns:
        cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].dropna()
    else:

        if 'Timestamp' in cgm.columns:
            cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = cgm['Timestamp'].dropna()
            if cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].str.contains(' ').all():
                print('true')
            else:
                if 'Time' in cgm.columns:
                    cgm = cgm[pd.notnull(cgm['Date'])]
                    cgm['Date'] = cgm['Date'].astype('str').str.split(' ').str[0]
                    cgm['Time'] = cgm['Time'].astype('str').str.split(' ').str[0]
                    cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(cgm['Date'] + "  "+ cgm['Time'])
                
        elif 'EventDateTime' in cgm.columns:
            cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = cgm['EventDateTime'].dropna()
        elif 'GlucoseDisplayTime' in cgm.columns:
            cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = cgm['GlucoseDisplayTime'].dropna()
        elif 'Time' in cgm.columns:
            cgm = cgm[pd.notnull(cgm['Date'])]
            cgm['Date'] = cgm['Date'].astype('str').str.split(' ').str[0]
            cgm['Time'] = cgm['Time'].astype('str').str.split(' ').str[0]
            cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(cgm['Date'] + "  "+ cgm['Time'])
           
    try:
        cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].str.replace('T',' '))
    except:
        pass
    if 'Glucose Value (mg/dL)' not in cgm.columns:
        if 'Glucose' in cgm.columns:
            cgm['Glucose Value (mg/dL)'] = cgm['Glucose']
        elif  'Historic Glucose'  in cgm.columns:
            cgm['Glucose Value (mg/dL)'] = cgm['Historic Glucose']
        elif  'Historic Glucose (mg/dL)'  in cgm.columns:
            cgm['Glucose Value (mg/dL)'] = cgm['Historic Glucose (mg/dL)']
            
        elif  'Sensor Glucose (mg/dL)'  in cgm.columns:
            cgm['Glucose Value (mg/dL)'] = cgm['Sensor Glucose (mg/dL)']
        elif 'GlucoseValue' in cgm.columns:
            cgm['Glucose Value (mg/dL)'] = cgm['GlucoseValue']

    cgm['Glucose Value (mg/dL)'] = cgm['Glucose Value (mg/dL)'].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))
    cgm = cgm.sort_values('Timestamp (YYYY-MM-DDThh:mm:ss)')
    cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'])
    griggs_cgm = cgm[['Timestamp (YYYY-MM-DDThh:mm:ss)','Glucose Value (mg/dL)']]

    griggs_cgm ['time_diff']= griggs_cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].diff().reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)']
    griggs_cgm ['time_diff']=griggs_cgm ['time_diff'].dt.round("5min")
    griggs_cgm ['time_diff']= griggs_cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].diff().reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)']
    griggs_cgm ['time_diff']= griggs_cgm ['time_diff'].dt.round("5min")
    griggs_cgm['Glucose Value (mg/dL)'] = griggs_cgm['Glucose Value (mg/dL)'].astype(str)
    
    idx = griggs_cgm[griggs_cgm.time_diff >'00:05:00'].index
    person_id = file.split('/')[-1].split('_')[0]
    if(len(idx)>0):
        for id in idx:
            griggs_cgm['Glucose Value (mg/dL)'] = griggs_cgm['Glucose Value (mg/dL)'].astype(str).replace('nan','NaN')
            m = griggs_cgm[griggs_cgm['Glucose Value (mg/dL)']!='NaN']['Glucose Value (mg/dL)'].astype(float).mean()
            first_slice = griggs_cgm[:id-1]
            second_slice = griggs_cgm[id-1:id+1]
            third_slice = griggs_cgm[id+1:]
            second_slice['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(second_slice['Timestamp (YYYY-MM-DDThh:mm:ss)'])
            second_slice = second_slice.set_index('Timestamp (YYYY-MM-DDThh:mm:ss)', drop=True)
            second_slice = second_slice.asfreq('5min',fill_value = m).reset_index().reset_index(drop=True)                    
            griggs_cgm['Glucose Value (mg/dL)'] = griggs_cgm['Glucose Value (mg/dL)'].astype(str).replace('NaN',m)
            second_slice['time_diff'] = pd.to_timedelta('00:05:00')
            griggs_cgm = pd.concat([first_slice,second_slice,third_slice])
            idx = griggs_cgm[griggs_cgm.time_diff > '00:05:00'].index
            if(len(idx)==0):
                break
    griggs_cgm['Glucose Value (mg/dL)'] = griggs_cgm['Glucose Value (mg/dL)'].astype(float).fillna(0.0).astype(int)
    griggs_cgm = griggs_cgm[griggs_cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'].notnull()]
    griggs_cgm = griggs_cgm.groupby(pd.Grouper(key='Timestamp (YYYY-MM-DDThh:mm:ss)', freq='5min')).sum().reset_index()
    
    ###########################################################################
    ###########################################################################
    ###########################################################################

    ################ Read ACT file ###########################################
    for file in glob.glob('Data/Circadian Data Export/*.csv'):
        num = file.split('/')[-1].split('_')[0]
        if ('00'+str(i) == num) or ('0'+str(i) == num) or (str(i) == num):
            act = pd.read_csv(file)
            break
    if len(act) == 0:
        continue
    idx = act.index.get_loc(act[act['Actiware Export File  (Version 05.00 )'] == '-------------------- Epoch-by-Epoch Data -------------------'].index[0])
    act.columns = list(act.iloc[idx+17:idx+18].iloc[0])
    act  = act.iloc[idx+19:]
    act = act.iloc[:, : 5]
    act['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime((act['Date']) + " "+ (act['Time']))
    griggs_act = act[['Timestamp (YYYY-MM-DDThh:mm:ss)','Activity']]
    griggs_act['Activity'] =  griggs_act['Activity'].astype(float)
    griggs_act ['time_diff']= griggs_act['Timestamp (YYYY-MM-DDThh:mm:ss)'].diff().reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)']
    griggs_act ['time_diff']=griggs_act ['time_diff'].dt.round("0.5min")
    griggs_act ['time_diff']= griggs_act['Timestamp (YYYY-MM-DDThh:mm:ss)'].diff().reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)']
    griggs_act ['time_diff']= griggs_act ['time_diff'].dt.round("0.5min")
    griggs_act['Activity'] = griggs_act['Activity'].astype(str)
    
    idx = griggs_act[griggs_act.time_diff >'00:00:30'].index
    person_id = file.split('/')[-1].split('_')[0]
    if(len(idx)>0):
        for id in idx:
            first_slice = griggs_act[:id-1]
            second_slice = griggs_act[id-1:id+1]
            third_slice = griggs_act[id+1:]
            m = griggs_act[griggs_act['Activity']!='NaN']['Activity'].astype(float).mean()
            second_slice['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(second_slice['Timestamp (YYYY-MM-DDThh:mm:ss)'])
            second_slice = second_slice.set_index('Timestamp (YYYY-MM-DDThh:mm:ss)', drop=True)
            second_slice = second_slice.asfreq('0.5min',fill_value = m).reset_index().reset_index(drop=True)                    
            second_slice['time_diff'] = list(first_slice['time_diff'])[1]
            griggs_act = pd.concat([first_slice,second_slice,third_slice])
            griggs_act['Activity'] = griggs_act['Activity'].astype(str).replace('nan',m)
            griggs_act['Activity'] = griggs_act['Activity'].astype(float).fillna(0.0).astype(int)
            idx = griggs_act[griggs_act.time_diff > '00:00:30'].index
            if(len(idx)<1):
                break
    try:
        griggs_act['Activity'] = griggs_act['Activity'].astype(float).fillna(0.0).astype(int)
    except:
        pass
    griggs_act = griggs_act.groupby(pd.Grouper(key='Timestamp (YYYY-MM-DDThh:mm:ss)', freq='5min')).sum().reset_index()

    ###########################################################################
    ###########################################################################
    ###########################################################################
    min_time_act = griggs_act.sort_values(by='Timestamp (YYYY-MM-DDThh:mm:ss)').reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)'][0]
    min_time_cgm = griggs_cgm.sort_values(by='Timestamp (YYYY-MM-DDThh:mm:ss)').reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)'][0]
    max_time_act = griggs_act.sort_values(by='Timestamp (YYYY-MM-DDThh:mm:ss)',ascending=False).reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)'][0]
    max_time_cgm = griggs_cgm.sort_values(by='Timestamp (YYYY-MM-DDThh:mm:ss)',ascending=False).reset_index()['Timestamp (YYYY-MM-DDThh:mm:ss)'][0]
  
    if min_time_act > min_time_cgm:
        griggs_cgm = griggs_cgm[griggs_cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] >= min_time_act]
    else:
        
        griggs_act = griggs_act[griggs_act['Timestamp (YYYY-MM-DDThh:mm:ss)'] >= min_time_cgm]

    if max_time_cgm > max_time_act:
        griggs_cgm = griggs_cgm[griggs_cgm['Timestamp (YYYY-MM-DDThh:mm:ss)'] <= max_time_act]
    else:
        griggs_act = griggs_act[griggs_act['Timestamp (YYYY-MM-DDThh:mm:ss)'] <= max_time_cgm]
    if len(griggs_act) ==0:
        continue
    s=[griggs_act.iloc[0][0].strftime('%d-%b-%y'),griggs_act.iloc[0][0].strftime('%H:%M'),'20','53','Person ID: '+str(person_id),'M']
    start = pd.DataFrame(s,columns=['ACT'])
    start.to_csv('start_sg_act.awd',index=False)
    df_act = griggs_act[['Activity']]
    df_act.columns = griggs_act[['Activity']].iloc[0]
    df_act = df_act[1:]
    df_act.to_csv('clock_sg_act.awd',index=False)
    data = data2 = "" 
    # Reading data from file1 
    with open('start_sg_act.awd') as fp: 
        data = fp.read() 
    with open('clock_sg_act.awd') as fp: 
        data2 = fp.read() 
    data += data2 
    try:
        os.makedirs('ACT')  
    except:
        pass# create directory [current_path]/feed/address
    with open (os.path.join('ACT/ACT_'+str(person_id)+'.awd'), 'w') as fp: 
        fp.write(data) 
        
    s=[griggs_cgm.iloc[0][0].strftime('%d-%b-%y'),griggs_cgm.iloc[0][0].strftime('%H:%M'),'20','53','Person ID: '+str(person_id),'M']
    start = pd.DataFrame(s,columns=['CGM'])
    start.to_csv('start_sg_cgm.awd',index=False)
    df_cgm = griggs_cgm[['Glucose Value (mg/dL)']]
    df_cgm.columns = griggs_cgm[['Glucose Value (mg/dL)']].iloc[0]
    df_cgm = df_cgm[1:]
    df_cgm.to_csv('clock_sg_cgm.awd',index=False)
    data = data2 = "" 
    # Reading data from file1 
    with open('start_sg_cgm.awd') as fp: 
        data = fp.read() 
    with open('clock_sg_cgm.awd') as fp: 
        data2 = fp.read() 
    data += data2 
    try:
        os.remove("start_sg_cgm.awd")
        os.remove("clock_sg_cgm.awd")
        os.remove("start_sg_act.awd")
        os.remove("clock_sg_act.awd")
    except:
        pass

    try:
        os.makedirs('CGM')  
    except:
        pass# create directory [current_path]/feed/address

    with open ('CGM/CGM_'+str(person_id)+'.awd', 'w') as fp: 
        fp.write(data) 
    print("completed ",person_id)

# matlab_loc = which('matlab')
# dir_path = os.path.dirname(os.path.realpath('wc_coherence.m'))
# subprocess.run(matlab_loc+" -sd \""+dir_path+"\" -r \"run('wc_coherence.m');exit\" -nojvm -nodisplay -nosplash")

completed  001
completed  002
completed  003
completed  004
completed  005
completed  006
completed  008
completed  009
completed  010
completed  011
completed  012
completed  013
completed  014
completed  015
completed  016
completed  017
completed  018
completed  020
completed  021
completed  022
completed  023
completed  024
completed  026
completed  027
completed  028
completed  029
completed  031
completed  032
completed  033
completed  034
completed  035
completed  036
completed  037
completed  038
completed  039
completed  041
completed  042
completed  043
completed  044
completed  045
completed  046


In [8]:
matlab_loc = "/Applications/MATLAB_R2020b.app/bin/matlab"
dir_path = os.path.dirname(os.path.realpath('wc_coherence.m'))
stream = os.popen('/Applications/MATLAB_R2020b.app/bin/matlab '+dir_path+'/wc_coherence.m')
output = stream.read()
output

#subprocess.getstatusoutput(f'/Applications/MATLAB_R2020b.app/bin/matlab '+dir_path+'/wc_coherence.m')
#os.system("/Applications/MATLAB_R2020b.app/bin/matlab < "+dir_path+"/wc_coherence.m;quit;")

'\n                                                                              < M A T L A B (R) >\n                                                                    Copyright 1984-2020 The MathWorks, Inc.\n                                                                R2020b Update 5 (9.9.0.1592791) 64-bit (maci64)\n                                                                                February 4, 2021\n\n \nTo get started, type doc.\nFor product information, visit www.mathworks.com.\n \n'